<a href="https://colab.research.google.com/github/JMQuinlan/Compressible/blob/main/Rayliegh_flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rayleigh flow calculations
# MAE 5140 - Compressible Flow

copyright 2021 - J. M. Quinlan  
University of Colorado  
Colorado Springs


Let's give this a try!

This piece of code is based on the pygasflow package, the documentation for which is found at:

https://pygasflow.readthedocs.io/en/latest/examples/tut-2.html#

And code (which you shoudn't need to get started, just `pip install` as below):

https://github.com/Davide-sd/pygasflow

In [1]:
# Note the ! before the command, which forces colab to run the code at the 
# command line rather than in python
#
# If you download to your personal computer, the installation command won't have
# the ! and may vary depending on how you installed python
#
! pip install pygasflow

     |████████████████████████████████| 74 kB 728 kB/s 


In [3]:
# Import the library
import pygasflow

Define some variables as an example

In [4]:
gamma = 1.4
M1 = 0.1

Find the required sonic reference conditions at state 1 and define a print function as shown in the pygasflow package documentation

In [15]:
def print_rayleigh(M, prs, drs, trs, tprs, ttrs, urs, eps):
    print("M \t\t {}".format(M))
    print("P/P* \t\t {}".format(prs))
    print("rho/rho* \t {}".format(drs))
    print("T/T* \t\t {}".format(trs))
    print("P0/P0* \t\t {}".format(tprs))
    print("T0/T0* \t\t {}".format(ttrs))
    print("U/U* \t\t {}".format(urs))
    print("(s*-s)/R \t {}".format(eps))
    print()

result = pygasflow.rayleigh_solver('m', M1, gamma=gamma)
print_rayleigh(*result)

M 		 0.1
P/P* 		 2.366863905325444
rho/rho* 	 42.24999999999999
T/T* 		 0.056020447463324116
P0/P0* 		 1.2591455962360274
T0/T0* 		 0.04677707363187564
U/U* 		 0.02366863905325444
(s*-s)/R 	 10.948700657848867



Specifically, what we first needed was $\frac{T_o}{T_o^*}$:

In [12]:
print("T0/T0* \t\t {}".format(result[5]))

T0/T0* 		 0.04677707363187564


This is the same value that we found from the table in the book: $\frac{T_o}{T_o^*} = 0.04678$

If we want to use values in further python calculations (not basically as a lookup table, as above), then returning the results as a python dictionary is probably a good idea:

In [18]:
result = pygasflow.rayleigh_solver('m', M1, gamma=gamma, to_dict=True)
print(result['ttrs'])

0.04677707363187564


Now, we're primed to solve our example problem.

Given out heat addition, we could find our difference in stagnation enthalpy.  Since we are also assuming that the gas is nonreacting and a calorically perfect gas (CPG), i.e., the specific heats are constant, the stagnation enthalpy equation may be written in terms of stagnation temperature.

$\frac{T_{o,e}}{T_{o,1}} = 1 + \frac{q}{c_p T_{o,1}}$

In [35]:
gamma = 1.4
M1 = 0.1
q = 2200.      # kJ/kg_{air flow}
c_p = 1.005    # kJ/(kg*K)
T_o1 = 300.    # K
P_o1 = 300.    # kPa

result = pygasflow.rayleigh_solver('m', M1, gamma=gamma, to_dict=True)

# e/1
ToRe1 = 1. + q/(c_p*T_o1)
print("T_oe/To1 \t {}".format(ToRe1))
# 1/star
ToR1s = result['ttrs']
print("T_o1/To* \t {}".format(ToR1s))

# So to get e/star = (e/1)*(1/star)
ToRes = ToRe1 * ToR1s
print("T_oe/To* \t {}".format(ToRes))

# Finally, use pygasflow to find the Mach number at the exit
result2 = pygasflow.rayleigh_solver('total_temperature_sub', ToRes, gamma=gamma, to_dict=True)

# Print the exit Mach number
print("M_exit \t\t {}".format(result2['m']))

T_oe/To1 	 8.296849087893865
T_o1/To* 	 0.04677707363187564
T_oe/To* 	 0.38810232069697154
M_exit 		 0.3223919681080784


We find the same thing that we did in class:

$M_\mathrm{exit} = 0.322$